In [ ]:
!pip install emnist
from emnist import extract_training_samples
from emnist import extract_test_samples

In [ ]:
X_train, y_train = extract_training_samples('letters')
X_test, y_test = extract_test_samples('letters')
X_train = X_train/255.
X_test = X_test/255.
# jedes Pixel in allen Bildern einen Wert zwischen 0 und 1 hat
print(X_train.shape)
print(X_test.shape)
# 124800 Bilder fürs Trainieren
# 20800 Bilder fürs Testen

In [ ]:
# Bilder von Training- und Test Datensatz zeigen 
import matplotlib.pyplot as plt

img_index_train = 8888 # das 8888. Image ausnehmen
img_train = X_train[img_index_train]
print("Training - Der Buchstabe Label: " + str(chr(y_train[img_index_train]+96)))

plt.imshow(img_train.reshape((28,28)), cmap='gray')

In [ ]:
img_index_test = 1888
img_test = X_test[img_index_test]
print("Test - Das Bild zeigt: " + str(chr(y_test[img_index_test]+96)))
plt.imshow(img_test.reshape((28,28)))

In [ ]:
import numpy as np
import random

n_train = X_train.shape[0]
n_test = X_test.shape[0]
image_shape = X_train[0].shape
n_classes = len(np.unique(y_train))
random.seed(4242)

print("Number of Training Images =", n_train)
print("Number of Test Images =    ", n_test)
print("Image Format =             ", image_shape)
print("Number of Classes =        ", n_classes)

In [ ]:
from keras.utils import np_utils

y_train_cat = np_utils.to_categorical(y_train-1, 26)
y_test_cat = np_utils.to_categorical(y_test-1, 26)

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print("Shape before = ", X_train.shape)
X_train_reshaped = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], 28, 28, 1)
print("Shape after  = ", X_train_reshaped.shape)

In [ ]:
BATCH_SIZE = 100
EPOCHS     = 40

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# Preparing data generators for data augmentation
train_datagen = ImageDataGenerator(shear_range=2, 
                                   zoom_range=0.2, 
                                   rotation_range=5, 
                                   width_shift_range=4, 
                                   height_shift_range=4, 
                                   featurewise_center=False, 
                                   samplewise_center=False, 
                                   featurewise_std_normalization=False, 
                                   samplewise_std_normalization=False)
# fit parameters from data
train_datagen.fit(X_train_reshaped)

train_generator = train_datagen.flow(X_train_reshaped, 
                                     y=y_train_cat, 
                                     batch_size=BATCH_SIZE)

In [ ]:
import keras
from keras import models, layers

dp = 0.2
class VGGNet(models.Sequential):
    def __init__(self, input_shape, nb_classes):
        super().__init__()
        self.add(Conv2D(32, (3, 3), padding='same',input_shape=input_shape))
        self.add(BatchNormalization())
        self.add(Activation('relu'))
        self.add(Conv2D(32, (3, 3), padding='same'))
        self.add(BatchNormalization())
        self.add(Activation('relu'))
        self.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        self.add(Dropout(dp))
        self.add(Conv2D(64, (3, 3), padding='same'))
        self.add(BatchNormalization())
        self.add(Activation('relu'))
        self.add(Conv2D(64, (3, 3), padding='same'))
        self.add(BatchNormalization())
        self.add(Activation('relu'))
        self.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        self.add(Dropout(dp))
        self.add(Conv2D(128, (3, 3), padding='same'))
        self.add(BatchNormalization())
        self.add(Activation('relu'))
        self.add(Conv2D(128, (3, 3), padding='same'))
        self.add(BatchNormalization())
        self.add(Activation('relu'))
        self.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        self.add(Dropout(dp))
        #-------------------------------Fully Connected Layer----------------------------------------
        self.add(Flatten(name='flatten'))
        self.add(Dense(128, name='fc1'))
        self.add(BatchNormalization())
        self.add(Activation('relu'))
        self.add(Dropout(dp))
        self.add(Dense(128, name='fc2'))
        self.add(BatchNormalization())
        self.add(Activation('relu'))
        self.add(Dropout(dp))
        self.add(Dense(26, name='fc3', activation ="softmax"))

In [ ]:
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, ELU
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU, AveragePooling2D

input_shape = X_train_reshaped[0].shape
model = VGGNet(input_shape, n_classes)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Training the model
history = model.fit(train_generator, 
                    steps_per_epoch=n_train/BATCH_SIZE,  
                    epochs=EPOCHS, 
                    verbose=1)

In [ ]:
score = model.evaluate(X_test_reshaped, y_test_cat)

print('Test Ergebniss')
print('Test loss:    ', score[0])
print('Test accuracy:', score[1])

In [ ]:
resultingClasses = np.argmax(model.predict(X_test_reshaped), axis=1)
incorrects = np.nonzero(resultingClasses != y_test-1)
print("Total number of misclassifications: ", len(incorrects[0]))

In [ ]:
plt.figure(figsize=(18,22))
index = 0
for imageNumber in incorrects[0]:
    #print(imageNumber)
    plt.subplot(20,20,index+1)
    plt.imshow(X_test[imageNumber], cmap='gray')
    plt.title("{} as {}".format(chr(y_test[imageNumber]+96), chr(resultingClasses[imageNumber]+96)))
    plt.xticks([])
    plt.yticks([])
    index = index + 1
    if index >= 400:
        break